In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
qdrant_api_key = os.getenv("qdrant_api_key")
gemini_api_key = os.getenv("gemini_api_key")

# Load model embedding

In [64]:

import torch
import tqdm


from sentence_transformers import SentenceTransformer
from tqdm import tqdm

class BGEEmbedder:
    def __init__(self, model_name="BAAI/bge-m3"):
        self.model = SentenceTransformer(model_name)
        self.prefix = "Represent this sentence for searching relevant passages: "

    def embed(self, texts, batch_size=16):
        # BGE-M3 yêu cầu prefix cho truy vấn và văn bản
        texts_with_prefix = [
            text if text.startswith(self.prefix) else self.prefix + text
            for text in texts
        ]

        # Dùng encode với batch size và normalize sẵn
        embeddings = self.model.encode(
            texts_with_prefix,
            batch_size=batch_size,
            normalize_embeddings=True,
            show_progress_bar=True
        )
        return embeddings




In [2]:
# from sentence_transformers import SentenceTransformer

# # Load model BGE-M3
# model = SentenceTransformer("BAAI/bge-m3")

# def embed_fn(text: str):
#     # BGE-M3 khuyến nghị prefix truy vấn bằng "Represent this sentence for searching relevant passages:"
#     if not text.startswith("Represent"):
#         text = "Represent this sentence for searching relevant passages: " + text
#     return model.encode(text, normalize_embeddings=True)


# Gọi Qdrant vector DB

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from qdrant_client.models import Filter
# api=qdrant_api_key
# client = QdrantClient(
#     url=r"https://0f47d391-b7c1-45d9-a956-5f7228cd80f3.europe-west3-0.gcp.cloud.qdrant.io:6333",
#     api_key=api,
#     prefer_grpc=False
# )
from qdrant_client import QdrantClient

client = QdrantClient(
    host="localhost",
    port=6333
)


# client.recreate_collection(
#     collection_name="flm_fap",
#     vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
# )
# client.delete(
#     collection_name="flm_fap",
#     points_selector=Filter(must=[])  # Xoá toàn bộ points
# )


In [ ]:
# from qdrant_client import QdrantClient
# from qdrant_client.http.models import Distance, VectorParams
# from tqdm import tqdm

# # === CONFIGURATION ===
# CLOUD_URL = "https://0f47d391-b7c1-45d9-a956-5f7228cd80f3.europe-west3-0.gcp.cloud.qdrant.io:6333"
# CLOUD_API_KEY = api
# COLLECTION_NAME = "flm_fap"
# LOCAL_HOST = "localhost"
# LOCAL_PORT = 6333

# # === 1. Connect to Cloud and Local ===
# cloud_client = QdrantClient(
#     url=CLOUD_URL,
#     api_key=CLOUD_API_KEY
# )

# local_client = QdrantClient(
#     host=LOCAL_HOST,
#     port=LOCAL_PORT
# )

# # === 2. Get schema (vector size + distance) ===
# from qdrant_client.http.models import Distance, VectorParams

# VECTOR_SIZE = 1024  # Hoặc đúng dimension bạn dùng
# DISTANCE = Distance.COSINE  # Hoặc Distance.DOT, Distance.EUCLID

# # # Tạo lại collection local
# # local_client.recreate_collection(
# #     collection_name=COLLECTION_NAME,
# #     vectors_config=VectorParams(
# #         size=VECTOR_SIZE,
# #         distance=DISTANCE
# #     )
# # )


# # === 4. Scroll all data from cloud and upsert to local ===
# BATCH_SIZE = 256
# offset = None
# total = 0

# print("[+] Copying vectors...")
# with tqdm() as pbar:
#     while True:
#         points, offset = cloud_client.scroll(
#             collection_name=COLLECTION_NAME,
#             limit=BATCH_SIZE,
#             with_payload=True,
#             with_vectors=True,
#             offset=offset
#         )
#         if not points:
#             break

#         from qdrant_client.http.models import PointStruct

#         converted_points = [
#             PointStruct(id=point.id, vector=point.vector, payload=point.payload)
#             for point in points
#         ]

#         local_client.upsert(
#             collection_name=COLLECTION_NAME,
#             points=converted_points
#         )

        
#         total += len(points)
#         pbar.update(len(points))

# print(f"[✓] Done! Copied {total} vectors to local Qdrant.")


[+] Copying vectors...


28784it [07:15, 66.07it/s]


KeyboardInterrupt: 

In [90]:

local_client.count(collection_name="flm_fap", count_filter=None)


CountResult(count=5852)

In [68]:
embedder = BGEEmbedder()  

# Demo search

## Tạo subject map cho các môn học

In [69]:
import pandas as pd
df_flm=pd.read_csv(r'D:\Learn\Kì 5\SEG301\Fap-Chat\data\DATA cố định\FLM\FINAL\FINAL_DF_FLM.csv')
subject_map = {
    row["SubjectCode"]: f"{row["SubjectCode"]} - {row["Subject Name"]}"
    for _, row in df_flm[["SubjectCode", "Subject Name"]].dropna().drop_duplicates().iterrows()
}
subject_embeddings = {
    code: embedder.embed([name])[0]
    for code, name in subject_map.items()
}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Xoá trong subject_map
subject_map = {
    k: v for k, v in subject_map.items()
    if not k.startswith("PHE_COM")
}

# Xoá trong embeddings
subject_embeddings = {
    k: v for k, v in subject_embeddings.items()
    if not k.startswith("PHE_COM")
}


## Đây là hàm để chọn ra top các môn học liên quan dựa vào việc so sánh querry với các subject map mới tạo ở trên

ae thấy có cái top_k là top các môn học nên trả ra

In [71]:
from numpy import dot
from numpy.linalg import norm
import heapq


def detect_subject(query, top_k=2, threshold=0.75):
    query_vec = embedder.embed([query])[0]
    
    sims = {
        code: dot(query_vec, emb) / (norm(query_vec) * norm(emb))
        for code, emb in subject_embeddings.items()
    }

    top_subjects = heapq.nlargest(top_k, sims.items(), key=lambda x: x[1])

    # Chỉ giữ lại những cái vượt ngưỡng
    top_subjects = [(code, score) for code, score in top_subjects if score >= threshold]

    # Nếu không có gì đạt ngưỡng → trả về danh sách rỗng
    return top_subjects



## Hàm dịch

In [95]:
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

  # => "I am learning natural language processing."

# def translate_vi_to_en_google(text):
#     return GoogleTranslator(source='auto', target='en').translate(text)
def translate_vi_to_en_google(text):
    translator = pipeline("translation", model=model, tokenizer=tokenizer, src_lang="vie", tgt_lang="eng_Latn")

    output = translator(text)
    
    return output[0]['translation_text']

## Type_map đây ae

In [137]:
TYPE_DESCRIPTIONS = {
    "overview": "general overview of the subject, goals, credits, syllabus, semester, prerequisite, Any subject",
    "construtive_question": "thought-provoking questions",
    "assessment": "evaluations, types of tests, exams, and grading weights for the subject",
    "session": "lecture sessions, lessons, topics covered in each week or session",
    "material": "recommended textbooks, reference materials, slides, or other learning resources",
    "learning outcome": "learning outcome, expected knowledge, skills, or competencies students should achieve after completing the course",
    "student_list": "list of students enrolled in the course, including name, student ID, and email"
}

TYPE_KEYWORDS = {
    "overview": ["overview", "objective", "goal", "credits", "semester", "pre-requisite", "syllabus", "any"],
    "construtive_question": ["why", "how", "what if", "critical", "discussion", "reflect", "ethical"],
    "assessment": ["exam", "test", "quiz", "grading", "project", "evaluation", "score"],
    "session": ["week", "lesson", "lecture", "topic", "schedule", "session", "class"],
    "material": ["textbook", "slide", "document", "reading", "reference", "material", "resource"],
    "learning outcome": ["learn", "outcome", "skill", "competency", "ability", "achieve", "knowledge"],
    "student_list": ["student", "name", "id", "mssv", "email", "class list", "enrolled", "danh sách sinh viên"]
}
type_embeddings = {
    t: embedder.embed([desc])[0] for t, desc in TYPE_DESCRIPTIONS.items()
}

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Hàm chạy chính

In [130]:
from numpy import dot
from numpy.linalg import norm
from collections import defaultdict
import re
# from rank_bm25 import BM25Okapi

# # Tokenize mô tả
# type_docs = [desc.lower().split() for desc in TYPE_DESCRIPTIONS.values()]
# bm25 = BM25Okapi(type_docs)
# type_list = list(TYPE_DESCRIPTIONS.keys())



# # --- Step 0: Optional Translate Query (nếu cần)
# query = "tổng quan môn search engine"

# # Nếu có hàm dịch: query_en = translate_vi_to_en_nllb(query)
# query_en = query  # nếu bạn không cần dịch hoặc đã là tiếng Anh

# --- Step 1: Detect type bằng embedding
# def detect_type_by_embedding(query_en):
#     query_vec = embedder.embed([query_en])[0]
#     sims = {
#         t: dot(query_vec, vec) / (norm(query_vec) * norm(vec))
#         for t, vec in type_embeddings.items()
#     }
#     print(sims)
#     best_type = max(sims, key=sims.get)

#     return best_type 
def detect_type_by_embedding(query_en, alpha=0.8, beta=0.2):
    query_vec = embedder.embed([query_en])[0]
    
    # 1. Embedding similarity
    sims = {
        t: dot(query_vec, vec) / (norm(query_vec) * norm(vec))
        for t, vec in type_embeddings.items()
    }

    # 2. Keyword score
    keyword_scores = defaultdict(int)
    query_lower = query_en.lower()
    for t, keywords in TYPE_KEYWORDS.items():
        for kw in keywords:
            if re.search(rf"\b{re.escape(kw)}\b", query_lower):
                keyword_scores[t] += 1

    # 3. Normalize keyword scores
    max_kw = max(keyword_scores.values(), default=1)
    keyword_scores_norm = {
        t: keyword_scores[t] / max_kw if max_kw > 0 else 0
        for t in type_embeddings
    }

    # 4. Combine scores
    final_scores = {
        t: alpha * sims.get(t, 0) + beta * keyword_scores_norm.get(t, 0)
        for t in type_embeddings
    }

    print("Similarity scores:", sims)
    print("Keyword scores:", dict(keyword_scores))
    print("Final combined scores:", final_scores)

    best_type = max(final_scores, key=final_scores.get)
    return best_type
# def detect_type_by_embedding(query_en, alpha=0.8, beta=0.2):
#     # 1. Embedding similarity
#     query_vec = embedder.embed([query_en])[0]
#     sims = {
#         t: dot(query_vec, vec) / (norm(query_vec) * norm(vec))
#         for t, vec in type_embeddings.items()
#     }

#     # 2. BM25 keyword-based score
#     query_tokens = query_en.lower().split()
#     bm25_scores = bm25.get_scores(query_tokens)
#     max_bm25 = max(bm25_scores) if max(bm25_scores) > 0 else 1
#     bm25_scores_norm = {type_list[i]: bm25_scores[i] / max_bm25 for i in range(len(type_list))}

#     # 3. Kết hợp
#     final_scores = {
#         t: alpha * sims.get(t, 0) + beta * bm25_scores_norm.get(t, 0)
#         for t in type_list
#     }

#     print("Cosine scores:", sims)
#     print("BM25 scores:", bm25_scores_norm)
#     print("Combined scores:", final_scores)

#     best_type = max(final_scores, key=final_scores.get)
#     return best_type

# query_type = detect_type_by_embedding(query_en)


query = "môn nào liên quan tới xử lí hình ảnh"
# query_en = query  # hoặc dùng translate nếu cần
query_en = translate_vi_to_en_google(query)
print('I/--------------Querry sau khi dịch:---------------')
print(query_en)

print('II/--------------Xác định chủ đề của querry---------------')
detected_type = detect_type_by_embedding(query_en)
print('III/--------------Xác định môn học của querry (nếu có)---------------')
detected_subject = detect_subject(query_en)

query_vec = embedder.embed([query])[0]


# query_filter = {"must": []}
# if detected_type:
#     query_filter["must"].append({"key": "type", "match": {"value": detected_type}})
# if detected_subject:
#     query_filter["must"].append({"key": "subject_code", "match": {"value": detected_subject}})

query_filter = {"should": [], "must":[]}
if detected_type:
    print("Loại detect được là:")
    print(detected_type)
    query_filter["must"].append({"key": "type", "match": {"value": detected_type}})
if detected_subject:
    for subject_code in detected_subject:
        print(subject_code[0])
        query_filter["should"].append({
            "key": "subject_code",
            "match": {"value": subject_code[0]}
        })

hits = client.search(
    collection_name="flm_fap",
    query_vector=query_vec.tolist(),
    limit=10,
    query_filter=query_filter if query_filter["must"] or query_filter["should"] else None
)


# --- Step 4: Hiển thị kết quả
for i, hit in enumerate(hits, 1):  # bắt đầu từ 1
    print(f"\n⭐ Top {i}")
    print(f"🔍 Score: {hit.score:.4f}")
    print(f"📘 Subject: {hit.payload.get('subject_code')} - {hit.payload.get('type')}")
    print(f"📄 Content:\n{hit.payload.get('content')}")
    print("--------")


I/--------------Querry sau khi dịch:---------------
Any subject related to image processing.
II/--------------Xác định chủ đề của querry---------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity scores: {'overview': 0.64287305, 'construtive_question': 0.6045393, 'assessment': 0.6490637, 'session': 0.6439435, 'material': 0.6581179, 'learning outcome': 0.5622215, 'student_list': 0.5417621}
Keyword scores: {'overview': 1, 'construtive_question': 0, 'assessment': 0, 'session': 0, 'material': 0, 'learning outcome': 0, 'student_list': 0}
Final combined scores: {'overview': 0.7142984390258789, 'construtive_question': 0.4836314201354981, 'assessment': 0.5192509651184082, 'session': 0.515154790878296, 'material': 0.526494312286377, 'learning outcome': 0.44977722167968753, 'student_list': 0.4334096908569336}
III/--------------Xác định môn học của querry (nếu có)---------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Loại detect được là:
overview

⭐ Top 1
🔍 Score: 0.5514
📘 Subject: CPV301 - overview
📄 Content:
TYPE: overview
Subject Code: CPV301
Subject Name: Computer Vision_Thị giác máy tính
Degree Level: Bachelor | Credits: 3 | Semester: 4
Pre-requisites: PFP 191, CSD 203
Scoring Scale: 10.0 | Min Avg Mark to Pass: 5.0
Approved: True on 11/24/2022
Subject Link: https://flm.fpt.edu.vn/gui/role/student/Syllabuses.aspx?sub Code=CPV 301&curriculum ID=2347

--- TIME ALLOCATION ---
45 h (60 sessions) contact hours + 1 h final exam + 104 h self-study
---TIME ALLOCATION END ---

Note:
- None

--------

⭐ Top 2
🔍 Score: 0.5427
📘 Subject: NLP301c - overview
📄 Content:
TYPE: overview
Subject Code: NLP301c
Subject Name: Natural Language Processing_Xử lý ngôn ngữ tự nhiên
Degree Level: Bachelor | Credits: 3 | Semester: 6
Pre-requisites: AIL 30 x and (PRP 201 c or PFP 191)
Scoring Scale: 10.0 | Min Avg Mark to Pass: 5.0
Approved: True on 4/9/2025
Subject Link: https://flm.fpt.edu.vn/gui/role/student/Syllabuses

In [ ]:
# --- CÁC IMPORT CẦN THIẾT ---
from numpy import dot
from numpy.linalg import norm
from collections import defaultdict
import re
import requests
import json
import time

# --- HÀM TỔNG HỢP VÀ TÓM TẮT BẰNG GEMINI ---
def summarize_with_gemini_combined(contents: list, query: str, api_key: str, model: str = "models/gemini-2.0-flash") -> str:
    """
    Tổng hợp và tóm tắt danh sách nội dung thành một câu trả lời ngắn gọn, chính xác cho query.
    """
    url = f"https://generativelanguage.googleapis.com/v1beta/{model}:generateContent?key={api_key}"
    headers = {"Content-Type": "application/json"}
    
    # Gộp tất cả nội dung từ các hits thành một chuỗi
    combined_content = "\n\n".join([content for content in contents])
    
    prompt = f"""
    Bạn là một trợ lý AI về trường đại học FPT. Hãy đọc và phân tích danh sách nội dung sau, sau đó đưa ra một câu trả lời đầy đủ bằng tiếng Việt để trả lời chính xác câu hỏi: "{query}".
    
    NỘI DUNG:
    ---
    {combined_content}
    ---

    CÂU TRẢ LỜI: Bắt đầu câu trả lời đừng có Dựa vào thông tin cung cấp
    """
    
    data = {"contents": [{"parts": [{"text": prompt}]}]}
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        summary = result["candidates"][0]["content"]["parts"][0]["text"]
        return summary.strip()
    except requests.exceptions.RequestException as e:
        print(f"Lỗi gọi API: {e}")
        return "Lỗi khi gọi API tóm tắt."
    except (KeyError, IndexError) as e:
        print(f"Lỗi khi xử lý response từ API: {e}. Response: {response.text}")
        return "Không thể tóm tắt nội dung do lỗi xử lý response."


def search(querry=''):
    # --- Chạy RAG -------------
    query = querry
    # query_en = query  # hoặc dùng translate nếu cần
    query_en = translate_vi_to_en_google(query)
    print('I/--------------Querry sau khi dịch:---------------')
    print(query_en)

    print('II/--------------Xác định chủ đề của querry---------------')
    detected_type = detect_type_by_embedding(query_en)
    print('III/--------------Xác định môn học của querry (nếu có)---------------')
    detected_subject = detect_subject(query_en)

    query_vec = embedder.embed([query])[0]


    # query_filter = {"must": []}
    # if detected_type:
    #     query_filter["must"].append({"key": "type", "match": {"value": detected_type}})
    # if detected_subject:
    #     query_filter["must"].append({"key": "subject_code", "match": {"value": detected_subject}})

    query_filter = {"should": [], "must":[]}
    if detected_type:
        print("Loại detect được là:")
        print(detected_type)
        query_filter["must"].append({"key": "type", "match": {"value": detected_type}})
    if detected_subject:
        for subject_code in detected_subject:
            print(subject_code[0])
            query_filter["should"].append({
                "key": "subject_code",
                "match": {"value": subject_code[0]}
            })

    hits = client.search(
        collection_name="flm_fap",
        query_vector=query_vec.tolist(),
        limit=50,
        query_filter=query_filter if query_filter["must"] or query_filter["should"] else None
    )



    # --- TỔNG HỢP KẾT QUẢ TÌM KIẾM VÀ ĐƯA RA CÂU TRẢ LỜI CHÍNH XÁC NHẤT ---
    
    print("\n==============================================")
    print("✅ KẾT QUẢ TỔNG HỢP CHO QUERY BẰNG GEMINI ✅")
    print("==============================================")
    if not hits:
        print("Không tìm thấy kết quả phù hợp.")
    else:
        print("\n🔍 CHI TIẾT CÁC KẾT QUẢ TÌM KIẾM:")
        for i, hit in enumerate(hits, 1):
            print(f"\n⭐ Top {i}")
            print(f"🔍 Score: {hit.score:.4f}")
            print(f"📘 Subject: {hit.payload.get('subject_code')} - {hit.payload.get('type')}")
            
            original_content = hit.payload.get('content', '')
            print(f"📄 Content Gốc: {original_content}...")
       
        print("------------------------------------------")

        # [TÙY CHỌN] Nếu bạn vẫn muốn hiển thị từng kết quả riêng lẻ với tóm tắt như trước
         # Lấy tất cả nội dung từ hits
        hit_contents = [hit.payload.get('content', '') for hit in hits]
        
        # Gọi Gemini API để tổng hợp và đưa ra câu trả lời chính xác nhất
        print("[*] Gemini đang tổng hợp và trả lời...")
        final_answer = summarize_with_gemini_combined(
            contents=hit_contents,
            query=query,  # Query gốc: "bạn cho tôi biết môn seg dạy cái gì"
            api_key=gemini_api_key
        )
        
        print(f"✨ Câu trả lời cuối cùng: {final_answer}")
        
        # print("[*] Gemini đang tóm tắt...")
        # summary = summarize_with_gemini_combined(
        #     contents=[original_content],  # Tóm tắt riêng từng hit
        #     query=query,
        #     api_key=gemini_api_key
        # )
        # # print(f"✨ Tóm tắt của Gemini: {summary}")
        # print("------------------------------------------")
        
        # # Tạm dừng 4 giây để tránh rate limit (trừ lần cuối)
        # if i < len(hits):
        #     time.sleep(4)

In [150]:
search('các môn học ở semester 5')

I/--------------Querry sau khi dịch:---------------
The subjects of the 5th semester are discussed.
II/--------------Xác định chủ đề của querry---------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity scores: {'overview': 0.6417157, 'construtive_question': 0.5791711, 'assessment': 0.62764615, 'session': 0.681198, 'material': 0.6172932, 'learning outcome': 0.5694497, 'student_list': 0.578665}
Keyword scores: {'overview': 1, 'construtive_question': 0, 'assessment': 0, 'session': 0, 'material': 0, 'learning outcome': 0, 'student_list': 0}
Final combined scores: {'overview': 0.713372564315796, 'construtive_question': 0.4633368968963623, 'assessment': 0.5021169185638428, 'session': 0.5449584007263184, 'material': 0.49383454322814946, 'learning outcome': 0.455559778213501, 'student_list': 0.46293201446533205}
III/--------------Xác định môn học của querry (nếu có)---------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Loại detect được là:
overview

✅ KẾT QUẢ TỔNG HỢP CHO QUERY BẰNG GEMINI ✅

🔍 CHI TIẾT CÁC KẾT QUẢ TÌM KIẾM:

⭐ Top 1
🔍 Score: 0.5564
📘 Subject: ENW493c - overview
📄 Content Gốc: TYPE: overview
Subject Code: ENW493c
Subject Name: Research Methods & Academic Writing Skills_Phương pháp nghiên cứu & Kỹ năng viết học thuật
Degree Level: Bachelor | Credits: 3 | Semester: 7
Pre-requisites: None
Scoring Scale: 10.0 | Min Avg Mark to Pass: 5.0
Approved: True on 4/9/2025
Subject Link: https://flm.fpt.edu.vn/gui/role/student/Syllabuses.aspx?sub Code=ENW 493 c&curriculum ID=2347

--- TIME ALLOCATION ---
Study hour (150 h) = 58 h online + 3 h offline + 100' PE + 87.3 h self-study
---TIME ALLOCATION END ---

Note:
- 1. Complete the online courses and get certifications to be allowed to take Final Exam 2. Final Exam included Practical Exam (PE): 100%3. Students who complete all MOOCs before the course deadline will receive 1 bonus point; otherwise, no bonus point will be awarded.4. Completion Criteri